In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
headers = {"Accept-Language": "en-US, en;q=0.5"}
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\yfird\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [8]:
pages=np.arange(1,8)
Name=[]
Price=[]
for page in pages:
    urls=["https://hubmart.com/product-category/groceries/page/{}/".format(page),"https://hubmart.com/product-category/beverages-groceries/page/{}/".format(page),"https://hubmart.com/product-category/fresh-food/page/{}/".format(page)]
    for url in urls:
        driver.get(url)
        results = driver.page_source
        soup = BeautifulSoup(results,"lxml")

        products=soup.find_all('div', class_="product-inner product-item__inner")
        for item in products:
            name=item.find('h2', class_="woocommerce-loop-product__title").text.strip()
            Name.append(name)
            #print(Name)
            #price
            price=item.find('span',class_="woocommerce-Price-amount amount")
            if price is not None:
                 price= price.text.strip()      
            else:
                 price = None
            Price.append(price)
            #print(Price)




In [16]:
items=pd.DataFrame({
    'Name':Name,
    'Price':Price
})

In [17]:
items.head()

,Name,Price
0,EVA WATER 1.5L,₦120
1,FUNTIME COCONUT CHIPS 30G,₦85
2,NESTLE PURE LIFE WATER 150CL,₦110
3,NESTLE PURE LIFE WATER 60CL,₦65
4,EVA WATER 50CL,₦65


In [18]:
items.shape

(372, 2)

In [29]:
items.isnull().any().sum()

0

In [22]:
items.dropna(axis=0)

,Name,Price
0,EVA WATER 1.5L,₦120
1,FUNTIME COCONUT CHIPS 30G,₦85
2,NESTLE PURE LIFE WATER 150CL,₦110
3,NESTLE PURE LIFE WATER 60CL,₦65
4,EVA WATER 50CL,₦65
...,...,...
367,BLACK & DECKER 0.6L AUTO RICE COOKER RC650-B5,"₦13,965"
368,Chicken pie,₦450
369,EVA WATER 1.5L,₦120
370,FUNTIME COCONUT CHIPS 30G,₦85


In [31]:
items.duplicated().any().sum()

1

In [27]:
nan_value=float("NaN")
items.replace("", nan_value, inplace=True)

In [28]:
items.dropna(subset = ["Price"], inplace=True)

In [35]:
items.drop_duplicates(subset=['Name'], keep='last',inplace=True)

In [36]:
items.shape


(333, 2)

In [38]:
items.to_csv("Hubmart groceries.csv",index=False)